# Phantom Silhouette方式のデモ

## 必要な諸々のインポート

In [1]:
import librosa
import soundfile as sf
import pyworld as pw
import numpy as np
from PhantomSilhouette import *

In [2]:
import IPython.display

## 関数を定義

In [3]:
def convert(wav_side: np.ndarray, sr: int) -> np.ndarray:
    """
    phantom silhouette法で音声の変換
    
    Parameters
    ----------
    wav_side: np.ndarray
        片方の音声
    sr: int
        サンプルレート
    
    Returns
    ----------
    wav_out: np.ndarray
        変換した音声
    """
    wav = wav_side.astype(np.float64)
    f0, sp, ap = pw.wav2world(wav, sr)
    f0_out, sp_out = phantom_silhouette(f0, sp, sr)
    wev_out = pw.synthesize(f0_out, sp_out, ap, sr)
    return wev_out.astype(np.float32)[:wav.shape[0]]

## 実際に変換してみる

In [4]:
wav, sr = librosa.load('sample.wav')

In [5]:
IPython.display.Audio(wav, rate=sr)

In [6]:
wav = convert(wav, sr)

In [7]:
IPython.display.Audio(wav, rate=sr)